# 4. Localisation

Let's import the necessary libraries.

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import scipy as sp
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.interpolate import LinearNDInterpolator
from scipy.optimize import minimize

from pickle import dump, load
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Normalization

Let's load : 
1. Model

In [32]:
model = load_model("model.keras")

print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 256)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129,029 (504.02 KB)

 Trainable params: 43,009 (168.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 86,020 (336.02 KB)

None


2. The scaler

In [33]:
scaler = load(open("scaler.pkl", "rb"))

print(scaler)

StandardScaler()


3. The dataset

In [34]:
data = pd.read_csv("data.csv")

Since we figured out the a model perform the work of the following application :
$$
\begin{align*}
    f : \mathbb{R} &\rightarrow \mathbb{R}\\
    c &\mapsto d
\end{align*}
$$
where : 
- $c$ is a gradient of the concentration 
- $d$ is a gradient of the distance from the source
 
We can now apply regular localisation techniques to find the position of the gas source.


Like cell network or GPS localisation, we can use the same principle to localise a point in a 2D space. The idea is to have a set of points with known coordinates and distances to the unknown point. By using the model, we can estimate the coordinates of the unknown point.

The method used by these systems is called trilateration. The mathematics behind trilateration involves solving a system of equations derived from the distance formula in Euclidean space.

## Basic concept
Given $n$ known points (anchors) with coordinates $(x_i, y_i)$ and their respective distances $d_i$ to the unknown point $(x, y)$, the goal is to find the coordinates $(x, y)$.

## Distance formula
The distance between the unknown point $(x, y)$ and each known point $(x_i, y_i)$ is given by:

$$ d_i = \sqrt{(x - x_i)^2 + (y - y_i)^2} $$

System of Equations
For each known point, we can write an equation based on the distance formula:

$$ 
\begin{cases}
(x - x_1)^2 + (y - y_1)^2 = d_1^2 \\
(x - x_2)^2 + (y - y_2)^2 = d_2^2 \\
\vdots \\
(x - x_n)^2 + (y - y_n)^2 = d_n^2
\end{cases}
$$

# Linearization

To solve this system of nonlinear equations, we can linearize it by subtracting one equation from another. For example, subtracting the first equation from the second:

$$
(x - x_2)^2 + (y - y_2)^2 - (x - x_1)^2 - (y - y_1)^2 = d_2^2 - d_1^2
$$

Expanding and simplifying:

$$
x^2 - 2x x_2 + x_2^2 + y^2 - 2y y_2 + y_2^2 - x^2 + 2x x_1 - x_1^2 - y^2 + 2y y_1 - y_1^2 = d_2^2 - d_1^2
$$

$$
-2x (x_2 - x_1) - 2y (y_2 - y_1) = d_2^2 - d_1^2 - x_2^2 + x_1^2 - y_2^2 + y_1^2
$$

This can be rewritten as:

$$
-2x (x_2 - x_1) - 2y (y_2 - y_1) = C_{12}
$$

where $C_{12}$ is a constant derived from the known distances and coordinates.

## Matrix Form
For $n$ points, we can form a system of linear equations:

$$
A \begin{pmatrix} x \\ y \end{pmatrix} = B
$$

where $A$ is a matrix of coefficients and $B$ is a vector of constants.

## Optimization Approach

When there are more than three points, the system is over-determined, and we can use optimization techniques to find the best fit solution. The error function to minimize is:

$$
\text{Error} = \sum_{i=1}^{n} \left( \sqrt{(x - x_i)^2 + (y - y_i)^2} - d_i \right)^2
$$

This is the function used in the locate_trilateration function to find the coordinates ((x, y)) that minimize the sum of squared errors between the calculated and measured distances.


In [35]:
def locate_trilateration(positions, distances):
    """
    Trilateration algorithm to locate a point in 2D space given the positions of the anchors and the distances to them.
    """

    # Check if the input is valid
    if positions.shape[0] < 3 or distances.shape[0] < 3:
        raise ValueError("Trilateration requires at least 3 anchors.")
 
    if positions.shape[0] != distances.shape[0]:
        raise ValueError("The number of positions and distances must be the same.")

    def error_function(point, positions, distances):
        # Calculate the distances between the point and the anchors
        calculated_distances = np.linalg.norm(positions - point, axis=1)
    
        # Calculate the error between the calculated distances and the real distances
        errors = (calculated_distances - distances) ** 2

        # Return the sum of the errors
        return np.sum(errors) 

    initial_guess = np.mean(positions, axis=0)

    result = minimize(error_function, initial_guess, args=(positions, distances))

    return result.x

Select points spaced evenly in the area from the dataset that have the same diffusion coefficient, source position.